In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [4]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [5]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.SingleEncDualDecAutoencoder import SingleEncDualDecAutoencoder

In [6]:
import matplotlib.pyplot as plt

In [7]:
# from mask_generation import masked_sketch

In [16]:
batch_size = 128
latent_dims = 64
num_nodes = 17
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 1000
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.00002
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=8
dense_hidden2=4
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "single_encoder_dual_decoder_sqr_data"
variational=False
coupling = False
obj_bbx_conditioning = False

In [12]:
batch_train_loader, batch_val_loader = data_loading.load_data_obj_with_parts(obj_data_postfix = '_obj_boundary_sqr',
                                                              part_data_post_fix = '_scaled_sqr',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

C:\Users\user\miniconda3\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [13]:
import gc
gc.collect()

21

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [17]:
reconstruction_loss_arr = []
kl_loss_obj_arr = []
kl_loss_part_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

bbox_loss_thresold = 1.0
vae = SingleEncDualDecAutoencoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200,400], gamma=0.75)
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    IOU_weight_delta = torch.tensor([(1+epoch)/nb_epochs])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true
        adj_true = train_data.edge_index
        batch = train_data.batch
        
        # Reshape data
        X_obj_true = node_data_true[::num_nodes, 1:]
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, label_true , class_true, variational, coupling)
        
        node_data_pred = output[0]
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred)*torch.tensor([0.5]).cuda()),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=torch.cat([torch.unsqueeze(X_obj_pred, axis=1), node_data_pred], axis=1), 
                                            true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin, has_obj=True)
        node_loss = loss.node_loss(label_pred, label_true, True)
        
        kl_weight = klw[icoef]
        
        if kl_weight>0:
            reconstruction_loss = kl_loss_part*kl_weight + (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            
            
#     scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes-1, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[1:num_nodes,1:5]*label_true[1:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[1:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_part_arr.append(batch_kl_loss_part.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = y_true
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        X_obj_true = node_data_true[::num_nodes, 1:]
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_true, label_true , class_true, variational, coupling)
        
        node_data_pred = output[0]
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred)*torch.tensor([0.5]).cuda()),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=torch.cat([torch.unsqueeze(X_obj_pred, axis=1), node_data_pred], axis=1), 
                                            true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin, has_obj=True)
        node_loss = loss.node_loss(label_pred,label_true, True)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss_part*kl_weight + (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[1:num_nodes,1:5]*label_true[1:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[1:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
    if ((kl_loss_part_arr[-1]>0.5) and 
        (abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.05) and 
        (bbox_loss_arr[-1]<1.0) and (epoch>300)):
        icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

200
[1,   399] loss: 157.000
200
[2,   399] loss: 105.800
200
[3,   399] loss: 95.625
200
[4,   399] loss: 92.035
200
[5,   399] loss: 90.383
200
[6,   399] loss: 89.448
200
[7,   399] loss: 88.758
200
[8,   399] loss: 88.029
200
[9,   399] loss: 87.067
200
[10,   399] loss: 85.997
200
[11,   399] loss: 85.154
200
[12,   399] loss: 84.746
200
[13,   399] loss: 84.490
200
[14,   399] loss: 84.271
200
[15,   399] loss: 84.084
200
[16,   399] loss: 83.901
200
[17,   399] loss: 83.720
200
[18,   399] loss: 83.537
200
[19,   399] loss: 83.353
200
[20,   399] loss: 83.156
200
[21,   399] loss: 82.967
200
[22,   399] loss: 82.760
200
[23,   399] loss: 82.544
200
[24,   399] loss: 82.321
200
[25,   399] loss: 82.089
200
[26,   399] loss: 81.851
200
[27,   399] loss: 81.659
200
[28,   399] loss: 81.444
200
[29,   399] loss: 81.261
200
[30,   399] loss: 81.083
200
[31,   399] loss: 80.892
200
[32,   399] loss: 80.701
200
[33,   399] loss: 80.502
200
[34,   399] loss: 80.293
200
[35,   399] loss:

200
[278,   399] loss: 19.656
200
[279,   399] loss: 19.614
200
[280,   399] loss: 19.570
200
[281,   399] loss: 19.527
200
[282,   399] loss: 19.485
200
[283,   399] loss: 19.444
200
[284,   399] loss: 19.404
200
[285,   399] loss: 19.361
200
[286,   399] loss: 19.318
200
[287,   399] loss: 19.277
200
[288,   399] loss: 19.236
200
[289,   399] loss: 19.194
200
[290,   399] loss: 19.150
200
[291,   399] loss: 19.116
200
[292,   399] loss: 19.071
200
[293,   399] loss: 19.029
200
[294,   399] loss: 18.989
200
[295,   399] loss: 18.947
200
[296,   399] loss: 18.906
200
[297,   399] loss: 18.866
200
[298,   399] loss: 18.827
200
[299,   399] loss: 18.784
200
[300,   399] loss: 18.747
200
[301,   399] loss: 18.705
200
[302,   399] loss: 18.664
200
[303,   399] loss: 18.626
200
[304,   399] loss: 18.584
200
[305,   399] loss: 18.548
200
[306,   399] loss: 18.503
200
[307,   399] loss: 18.464
200
[308,   399] loss: 18.425
200
[309,   399] loss: 18.382
200
[310,   399] loss: 18.344
200
[311, 

[551,   399] loss: 10.727
200
[552,   399] loss: 10.711
200
[553,   399] loss: 10.700
200
[554,   399] loss: 10.673
200
[555,   399] loss: 10.650
200
[556,   399] loss: 10.635
200
[557,   399] loss: 10.613
200
[558,   399] loss: 10.597
200
[559,   399] loss: 10.574
200
[560,   399] loss: 10.562
200
[561,   399] loss: 10.549
200
[562,   399] loss: 10.518
200
[563,   399] loss: 10.504
200
[564,   399] loss: 10.481
200
[565,   399] loss: 10.469
200
[566,   399] loss: 10.450
200
[567,   399] loss: 10.427
200
[568,   399] loss: 10.412
200
[569,   399] loss: 10.393
200
[570,   399] loss: 10.376
200
[571,   399] loss: 10.362
200
[572,   399] loss: 10.339
200
[573,   399] loss: 10.315
200
[574,   399] loss: 10.297
200
[575,   399] loss: 10.286
200
[576,   399] loss: 10.265
200
[577,   399] loss: 10.252
200
[578,   399] loss: 10.240
200
[579,   399] loss: 10.216
200
[580,   399] loss: 10.196
200
[581,   399] loss: 10.179
200
[582,   399] loss: 10.160
200
[583,   399] loss: 10.150
200
[584,   39

KeyboardInterrupt: 

In [91]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = SingleEncDualDecAutoencoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true
    X_obj_true = node_data_true[::num_nodes, 1:]
    X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_true, label_true , class_true,  variational, coupling)
    node_data_pred_test = output[0]
#     node_data_pred_test = node_data_pred_test[:, 1:, :]
    X_obj_pred_test = output[1]
#     node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    X_obj_pred_test = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred_test)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred_test)*torch.tensor([0.5]).cuda()),
                              -1)
    pred_shape =  node_data_pred_test.shape
    node_data_true = torch.reshape(node_data_true, (pred_shape[0], pred_shape[1]+1, pred_shape[2]+1))
    res_dfs.append(metrics.get_metrics(node_data_true[:, 1:, :], X_obj_true, node_data_pred_test,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes-1,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

ValueError: operands could not be broadcast together with shapes (128,16) (128,15) 

In [89]:
node_data_true.shape

torch.Size([2176, 5])

In [ ]:
def inference(X_obj, part_decoder, obj_class, nodes, batch_size, latent_dims):
    
    nodes = torch.reshape(nodes,(batch_size,part_decoder.num_nodes))
    obj_class = torch.reshape(obj_class, 
                              (batch_size, num_classes))
    
    # obj sampling
    z_latent_obj = X_obj
    
    print(z_latent_obj.shape, obj_class.shape)
    conditioned_obj_latent = torch.cat([obj_class, z_latent_obj],dim=-1)
    conditioned_obj_latent = torch.cat([nodes, conditioned_obj_latent],dim=-1)

    # part sampling
    z_latent_part = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    conditioned_part_latent = torch.cat([conditioned_obj_latent, z_latent_part],dim=-1)
    
    x_bbx, _, _, _ = part_decoder(conditioned_part_latent)
    
    return x_bbx, X_obj
    

In [ ]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = True
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                        )


vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

part_decoder = vae.gcn_decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = inference(X_obj_true, part_decoder, class_true, label_true, batch_size, latent_dims)
    node_data_pred_test = output[0]
    X_obj_pred_test = output[1]
#     node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    
#     res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test,
#                                X_obj_pred_test,
#                                label_true,
#                                class_true,
#                                num_nodes,
#                                num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            image = plot_utils.plot_bbx((node_data_true[num_nodes*j:num_nodes*(j+1)
                                                       ,1:5]*label_true[num_nodes*j:num_nodes*(j+1)]).detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_utils.plot_bbx((node_data_pred_test[j]*label_true[num_nodes*j:num_nodes*(j+1)]).detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

In [ ]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',
                                                                   'IOU', 'MSE']]

In [ ]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

In [ ]:
res_obj_level[['IOU', 'MSE']].mean()

In [ ]:
X_obj_pred[:10,2:]-X_obj_pred[:10,:2]

In [ ]:
X_obj_true[:10, 2:]-X_obj_true[:10, :2]

In [ ]:
res_obj_level.to_csv(model_path+'/obj_metrics.csv')

In [ ]:
result.mean().reset_index()

In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)